In [1]:
include("../../src/Tsallis/rep.jl")
using Plots
using LaTeXStrings
using DataFrames, CSV

In [5]:
X0 = [-0.01, -0.01, -0.20, 0.8, 0.8]
mu_B = 10.0
eB = 0.13
q = 1.00001
Ts = 300.0:-2.0:10.0
Nodes = get_nodes(128, 20)
nodes = gauleg(0, 1, 128)
sol = zeros(length(Ts), length(X0)+1)
sol[:, 1] = Ts
for (i, T) in enumerate(Ts)
    if i % 10 == 0
        println("---------------------------Calculating T = ", T, " MeV; Step ", i, " of ", length(Ts))
    end
    NewX = Tmu(T/hc, mu_B/hc, X0, eB*(1000/hc)^2, q, Nodes, nodes)
    sol[i, 2:end] = NewX
    X0 = NewX
end

---------------------------Calculating T = 282.0 MeV; Step 10 of 146
---------------------------Calculating T = 262.0 MeV; Step 20 of 146
---------------------------Calculating T = 242.0 MeV; Step 30 of 146
---------------------------Calculating T = 222.0 MeV; Step 40 of 146
---------------------------Calculating T = 202.0 MeV; Step 50 of 146
---------------------------Calculating T = 182.0 MeV; Step 60 of 146
---------------------------Calculating T = 162.0 MeV; Step 70 of 146
---------------------------Calculating T = 142.0 MeV; Step 80 of 146
---------------------------Calculating T = 122.0 MeV; Step 90 of 146
---------------------------Calculating T = 102.0 MeV; Step 100 of 146
---------------------------Calculating T = 82.0 MeV; Step 110 of 146
---------------------------Calculating T = 62.0 MeV; Step 120 of 146
---------------------------Calculating T = 42.0 MeV; Step 130 of 146
---------------------------Calculating T = 22.0 MeV; Step 140 of 146


In [8]:
df1 = DataFrame(sol, [:T, :phi_u, :phi_d, :phi_s, :Phi1, :Phi2])
T1 = df1.T
phi1 = df1.phi_u
println(phi1[end:-1:end-10])

[-1.9236807383413983, -1.9236807383413983, -1.9236807383413983, -1.9236807383413983, -1.9236807383413983, -1.9236807383413983, -1.9236807383413983, -1.9236807383413983, -1.923680738341391, -1.923680738341326, -1.9236807383409322]


UndefVarError: UndefVarError: `sol` not defined in `Main`
Suggestion: check for spelling errors or missing imports.